In [1]:
import random
import copy

# GA-VNS Code

In [10]:
def swap(X):
    X = copy.deepcopy(X)
    i, j = random.sample(range(0, len(X)), 2)
    X[i], X[j] = X[j], X[i]
    return X
    
def ls_vns(X1, iteration):
    epoch = 0
    solution = copy.deepcopy(X1)
    
    while (epoch < iteration):
        candidate = swap(solution)
        if (count_fitness(candidate) > count_fitness(solution)):
            solution = copy.deepcopy(candidate)
        else:
            solution = solution
        epoch+=1
    return solution
        
def VNS(X, kmax, iteration):
    #Shaking
    k = 1
    Xt = copy.deepcopy(X)
    
    while (k <= kmax):
        #shaking
        X1 = random.sample(X, len(X))
        #local search
        X2 = copy.deepcopy(ls_vns(X1, iteration))

        #change neighborhood
        if (count_fitness(X2) > count_fitness(Xt)):
            Xt = copy.deepcopy(X2)
            k = 1
        else :
            k = k + 1
    #print("Solusi terbaik =", Xt, count_fitness(Xt))
    return Xt

In [14]:
# crossover
def crossover(parent1, parent2):
    pos = random.randint(0, len(parent1)-1)
    child1 = parent1[0:pos] + parent2[pos:]
    child2 = parent2[0:pos] + parent1[pos:]
    return [child1, child2]
 
# mutation
def mutation(parent):
    i, j = random.sample(range(0, len(parent)-1), 2)
    parent[i], parent[j] = parent[j], parent[i]
    child = parent
    return [child]

#count fitness
def count_fitness(individu):
    harga_beli = [3000, 2000, 1000, 5000, 2500, 3000, 3500, 1500]
    harga_jual = [3500, 2500, 1500, 6000, 3500, 4000, 4000, 2000]
    rawat_gdg = 300000
    biaya_tkerja = 20000
    kapasitas = 2000
    
    TC = sum([individu*harga_beli for individu,harga_beli in zip(individu,harga_beli)])+rawat_gdg+biaya_tkerja
    TR = sum([individu*harga_jual for individu,harga_jual in zip(individu,harga_jual)])
    fit_value = (TR-TC)-((sum(individu)-kapasitas)*1000)
    return fit_value
    
def set_fitness(individu): 
    fitness = []
    for i in range(0, len(individu)):
        fitness.append(count_fitness(individu[i]))
    return fitness

#selection
def selection(individu, popsize):
    fitness = set_fitness(individu)
    for i in range(0, len(individu)):
        individu[i].insert(0, fitness[i])
    individu = sorted(individu)
    for i in range(len(individu)):
        individu[i].pop(0)
    individu = individu[-popsize:]
    return individu

def get_best(individu):
    f = set_fitness(individu)
    best = max(f)
    index = f.index(best)
    print("Solusi terbaik =", individu[index], " fitness =", best)
        
# main GA
def GAVNS(data, chromosome, popsize, cr, mr, generation, kmax, iteration):
    individu = []
    fitness = []
    best_individu = []
    epoch = 0
    
    interval = generation/2
    
    #init individu
    for i in range(popsize):
        individu.append(random.choices(data, k=chromosome))
        
    while (epoch < generation):
        #crossover
        do_crossover = cr*popsize
        if (do_crossover>=2):
            for i in range(0, round(do_crossover/2)):
                individu.extend(crossover(individu[random.randint(0, popsize-1)], individu[random.randint(0, popsize-1)]))
        #mutation
        do_mutation = mr*popsize
        for i in range(0, round(do_mutation)):
            individu.extend(mutation(individu[random.randint(0, popsize)]))
        
        #do VNS di pertengahan dan di akhir generasi
        if (epoch==interval or epoch==generation-1):
            n_repair = len(individu) - popsize
            for i in range (0, n_repair):
                individu[popsize+1] = VNS(individu[popsize+1], kmax, iteration)
        
        #selection
        individu = selection(individu, popsize)
        epoch = epoch + 1
    print()
    get_best(individu)

# main code

In [16]:
data = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 1000, 1500, 2000]

#parameter
populationsize = 10
gen_length = 8
cr = 0.4
mr = 0.6
generation = 10
kmax = 5
iteration = 10

%time GAVNS(data, gen_length, populationsize, cr, mr, generation, kmax, iteration)


Solusi terbaik = [250, 100, 100, 400, 850, 900, 100, 250]  fitness = 1280000
Wall time: 37.9 ms
